### **4.7 - How to make your tasks dependent**

#### **`depends_on_past`**

Al principio del curso, hemos visto cómo podemos definir dependencias entre diferentes tareas. Hemos descubierto las funciones 'set_upstream' y 'set_downstream', así como los Python bitshift operators. Una cosa para tener en cuenta es que esas dependencias se aplican dentro del DAGRun actual. En este video rápido vamos a ver cómo usar **`'depends_on_past'`** y **`'wait_for_downstream'`** para crear dependencias de tareas entre el DAGRun actual y el anterior. Comencemos con el parámetro **`'depends_on_past'`**. Imagina que tienes un DAGRun con las tareas A, B y C. En la primera ejecución, todas las tareas terminaron con éxito. Por otro lado, en la segunda ejecución, la tarea B falló y por lo tanto la tarea C falló también, pero con el estado 'upstream_failed'. Luego, en la tercera ejecución, todas las tareas terminaron con éxito. En este ejemplo, puedes ver que cada DAGRun se ejecuta independientemente de los demás.

Lo que puede querer en cambio es evitar que se ejecute la tarea B en el tercer DAGRun si la tarea B ha fallado en la DAGRun anterior. Eso es lo que **`depends_on_past`** te permite definir. La tarea B actual, en el caso de utilizar depends_on_past, **NO TENDRÁ STATUS (estado)**.  **`Este parámetro se aplica a nivel de tarea`**, por lo que puede definir específicamente una tarea para que dependa de su instancia anterior, **o puede aplicar el parámetro a todas las tareas utilizando el diccionario** **`'default_args'`**. El alcance que desees depende realmente de tu caso de uso. Cuando la tarea con **`'depends_on_past'`** se ejecuta por primera vez, el parámetro no se aplica ya que no hay un pasado que buscar. Si la tarea ha fallado en el DAGRun anterior, el estado de la tarea en el DAGRun actual se desajusta.

<center><img src="https://i.postimg.cc/J0mqK6G4/a656.png"></center>

En el ejemplo siguiente, en el segundo DAGRun, python_task_2 ha fallado representado por el cuadrado rojo y la tarea posterior (downstream) bash_task_2 tiene el estado upstream_failed como se muestra por el cuadrado naranja. Ahora, si echas un vistazo al tercer DAGRun, la primera tarea ha terminado con éxito, pero la segunda tarea python_task_2, no tiene ningún estado como puedes ver por el cuadrado blanco, de hecho, ya que la tarea python_task_2 tiene 'depends_on_past' establecido en True y la tarea falló en DAGRun 2, no se activará en DAGRun 3. En resumen, si quieres ejecutar una tarea determinada sólo si esta tarea ha terminado con éxito en el DAGRun anterior, entonces establece 'depends_on_past' a True. Por cierto, este parámetro está establecido en False por defecto.

<center><img src="https://i.postimg.cc/qMvcRCKY/a657.png"></center>

___

#### **`wait_for_downstream`**

Ahora que hemos visto depends_on_past vamos a descubrir el segundo parámetro, **`'wait_for_downstream'`**. Al igual que **depends_on_past**, **wait_for_downstream** **`se define a nivel de tarea pero también se puede definir dentro del diccionario 'default_args' para que se aplique a todas las tareas`**. Este parámetro le permite imponer que una tarea X determinada, espere a que las tareas inmediatamente posteriores a su instancia anterior (DagRun anterior) terminen con éxito antes de ejecutarse. Repito que este parámetro le permite imponer que una tarea X determinada espere a que las tareas inmediatamente posteriores a su instancia anterior terminen con éxito antes de ejecutarse.

<center><img src="https://i.postimg.cc/D0Bq679B/a658.png"></center>

Si tomamos el ejemplo mostrado aquí, '**`wait_for_downstream`**' se aplica a la tarea bash_task _1. Debido a que la tarea python_task_2 todavía se está ejecutando en DAG Run 2 y esta tarea se encuentra a continuación (downstream) de la tarea bash_task_1, en DAGRun 3, bash_task_1 no se activa ya que espera a las tareas python_task_2 y bash_task_2 en DAGRun 2 finalicen. Puedes ver eso por el cuadrado blanco aquí. Como es la primera tarea que se ejecuta, todas las tareas no se inician también. 'wait_for_downstream' es realmente útil cuando tienes múltiples DAGRuns con la misma tarea X trabajando en el mismo activo (asset) y este activo es utilizado por tareas posteriores (downstream) a la tarea X.

<center><img src="https://i.postimg.cc/mrCCq3xc/a659.png"></center>

De hecho, en un entorno concurrente, no te gustaría tener el segundo DAGRun iniciado antes de que el primero haya terminado, y así tener la tarea X en el segundo DAGRun trabajando con datos que aún no han sido procesados por la tarea X en el primer DAGRun.

<center><img src="https://i.postimg.cc/ZKs8yKFq/a660.png"></center>